In [117]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import scipy.sparse
import pandas as pd
import numpy as np
import talib as ta
import xgboost as xgb

In [118]:
# APIキー、environmentはデモか本番か
accountID = "101-009-17701910-001"
access_token = "37e74db6605da64aeb23ec4105187cb9-6cb80b7e58440f6cac193f567e80c6c3"
api = API(access_token=access_token ,environment="practice")

In [119]:
# データ取得に使う定数
COUNT = 5000        # 一度に取得するデータ数(max:5000)
NB_ITR = 3          # count * NB_ITR 分データを取得
GRANULARITY = "M15"
INSTRUMENT = "USD_JPY"

In [120]:
# oandaAPIからデータを取得する
def get_candles(instrument="USD_JPY", params=None):
    """
        足データを取得してDataFrameに変換
    """
    instruments_candles = instruments.InstrumentsCandles(instrument=instrument, params=params)

    api.request(instruments_candles)
    response = instruments_candles.response # json型

    df = pd.DataFrame(response["candles"]) # response["candles"]はlist型

    return df

In [121]:
# 足データを取得するためにoandaAPIに渡すパラメータ
params = {
    "granularity": GRANULARITY,
    "count": COUNT,
    #"price": "B",
}

In [122]:
# 足データの取得
candles = None
for i in range(NB_ITR):
    new_candles = get_candles(instrument=INSTRUMENT, params=params)
    params["to"] = new_candles["time"].iloc[0]
    print(params["to"])
    candles = pd.concat([new_candles, candles])

candles

2020-11-03T13:00:00.000000000Z
2020-08-21T08:45:00.000000000Z
2020-06-10T05:45:00.000000000Z


,complete,volume,time,mid
0,True,1251,2020-06-10T05:45:00.000000000Z,"{'o': '107.509', 'h': '107.525', 'l': '107.389..."
1,True,924,2020-06-10T06:00:00.000000000Z,"{'o': '107.474', 'h': '107.505', 'l': '107.448..."
2,True,814,2020-06-10T06:15:00.000000000Z,"{'o': '107.452', 'h': '107.480', 'l': '107.422..."
3,True,1206,2020-06-10T06:30:00.000000000Z,"{'o': '107.424', 'h': '107.443', 'l': '107.380..."
4,True,955,2020-06-10T06:45:00.000000000Z,"{'o': '107.430', 'h': '107.432', 'l': '107.338..."
...,...,...,...,...
4995,True,531,2021-01-18T13:30:00.000000000Z,"{'o': '103.751', 'h': '103.766', 'l': '103.732..."
4996,True,662,2021-01-18T13:45:00.000000000Z,"{'o': '103.766', 'h': '103.766', 'l': '103.719..."
4997,True,749,2021-01-18T14:00:00.000000000Z,"{'o': '103.723', 'h': '103.754', 'l': '103.720..."
4998,True,462,2021-01-18T14:15:00.000000000Z,"{'o': '103.752', 'h': '103.758', 'l': '103.732..."


In [131]:
# midはbidとaskの間で、midにスプレッドを足したものがask、引いたものがbidって感じかな？？
# price_arrをseriesからlist型で{'o': '107.554', 'h': '107.558', 'l': '107.507', 'c': '107.538'}が時系列順に羅列される
price_arr = candles["mid"]
price_arr = list(price_arr)
price_arr[-1]


{'o': '103.736', 'h': '103.746', 'l': '103.728', 'c': '103.728'}

In [138]:
# 終値を抽出
close_price = [float(dict(i)["c"]) for i in price_arr]
len(close_price)
close_price[14997]



103.754

In [135]:
# 「cur_pos-period ~ cur_pos+1」の期間でmaを計算する
# 20日移動平均（20SMA)であれば、本日を含めた過去20日間の終値を合計し、日数の20で割って平均
def get_ma(price_arr, cur_pos, period = 20):
    if cur_pos <= period:
        s = 0
    else:
        s = cur_pos - period + 1
    tmp_arr = price_arr[s:cur_pos+1] # cur_posってなってたけど、+1しないとだめじゃない？？
    #tmp_arr.reverse()
    print(tmp_arr)
    prices = np.array(tmp_arr, dtype=float)
    return ta.SMA(prices, timeperiod = period)[-1]

In [139]:
#get_ma(close_price, 14999, 3)
